In [2]:
# %%
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from src.dataScaling import data_scaler

# %%
# dataPath = 'src/H2DB.h5'
dataPath = 'src/H2DB_L.h5'

ddOrg = dd.read_hdf(dataPath, key='c')
ddWdot = dd.read_hdf(dataPath, key='wdot')

# %%
input_features = [
    'H', 'H2', 'O', 'O2', 'OH', 'H2O', 'N2', 'HO2', 'H2O2', 'Hs', 'Temp'
]
labels = input_features

org = ddOrg.compute()
wdot = ddWdot.compute()


# %%
def read_h5_data(input_features, labels):
    in_scaler = data_scaler()
    out_scaler = data_scaler()
    input_df = org[input_features]
    input_np = in_scaler.fit_transform(input_df[input_features].values, 'std2')

    # label_df = ((new[labels]-old[labels]).div((org.dt+old.dt), axis=0))
    label_df = wdot[labels]
    label_np = out_scaler.fit_transform(label_df[labels].values, 'std2')

    return input_np, label_np, in_scaler, out_scaler


x_input, y_label, in_scaler, out_scaler = read_h5_data(
    input_features=input_features, labels=labels)
x_train, x_test, y_train, y_test = train_test_split(x_input,
                                                    y_label,
                                                    test_size=0.05)
pickle.dump((labels, in_scaler, out_scaler), open('./data/tmp.pkl', 'wb'))

#%%


In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if not tf.test.is_gpu_available():
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

%load_ext tensorboard


print("TensorFlow version: ", tf.__version__)

Found GPU at: /device:GPU:0
TensorFlow version:  2.0.0-dev20190715


In [3]:
# %%
import os
import pickle

import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow
import tensorflow.keras as keras
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.models import Model, Sequential

from src.dataScaling import data_scaler
from src.res_block import res_block
from src.utils import SGDRScheduler


# %%
print('set up ANN')

n_neuron = 100
scale = 3
branches = 3
fc=True

for n_neuron in [64]:
    for branches in [5]:
        for fc in [True]:
            m_name='n{}_b{}_fc{}'.format(n_neuron,branches,fc)
            dim_input = x_train.shape[1]
            dim_label = y_train.shape[1]

            batch_norm = False

            # strategy = tensorflow.distribute.MirroredStrategy(devices=["/gpu:0"])
            # print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

            # with strategy.scope():
            inputs = Input(shape=(dim_input,), name='input_1')
            x = Dense(n_neuron, activation='relu')(inputs)

            # less then 2 res_block, there will be variance
            x = res_block(x, scale, n_neuron, stage=1, block='a',
                        bn=batch_norm, branches=branches)
            x = res_block(x, scale, n_neuron, stage=1, block='b',
                        bn=batch_norm, branches=branches)
            # x = res_block(x, scale, n_neuron, stage=1, block='c', bn=batch_norm,branches=branches)

            if fc ==True:
                x = Dense(100, activation='relu')(x)
            # x = Dropout(0.1)(x)
            predictions = Dense(dim_label, activation='linear', name='output_1')(x)

            model =  Model(inputs=inputs, outputs=predictions)
            model.summary()

            loss_type = 'mse'
            model.compile(loss=loss_type, optimizer='adam', metrics=['accuracy'])

            # %%
            print('Training')
            batch_size = 1024*8*8
            epochs = 400
            vsplit = 0.1
            # model.compile(loss=loss_type, optimizer='adam', metrics=[coeff_r2])

            # checkpoint (save the best model based validate loss)
            !mkdir ./tmp
            filepath = "./tmp/{}.weights.best.cntk.hdf5".format(m_name)

            checkpoint = ModelCheckpoint(filepath,
                                        monitor='val_loss',
                                        verbose=1,
                                        save_best_only=True,
                                        mode='min',
                                        save_freq='epoch')
                                        # period = 10)

            epoch_size = x_train.shape[0]
            ep_size = 0
            base = 2
            clc = 2
            for i in range(2):
                ep_size += base*clc**(i)
            print(ep_size)
            epochs, c_len = ep_size, base
            schedule = SGDRScheduler(min_lr=1e-6, max_lr=1e-4,
                                    steps_per_epoch=np.ceil(epoch_size/batch_size),
                                    cycle_length=c_len, lr_decay=0.8, mult_factor=2)

            callbacks_list1 = [checkpoint, tensorflow.keras.callbacks.TensorBoard('./tb/{}'.format(m_name),histogram_freq=1, profile_batch = 3)]

            model.load_weights(filepath)

            # fit the model
            history = model.fit(
                x_train, y_train,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=vsplit,
                verbose=2,
                callbacks=callbacks_list1,
                shuffle=False
            )

set up ANN
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 11)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 64)           768         input_1[0][0]                    
__________________________________________________________________________________________________
res1a_branch2a (Dense)          (None, 192)          12480       dense[0][0]                      
__________________________________________________________________________________________________
res1a_branch2a_0 (Dense)        (None, 192)          12480       dense[0][0]                      
___________________________________________________________________________________

W0716 16:13:21.780362 140016731551552 deprecation.py:323] From /home/edison/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:460: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


Train on 16136997 samples, validate on 1793000 samples
Epoch 1/6

Epoch 00001: val_loss improved from inf to 0.00000, saving model to ./tmp/n64_b5_fcTrue.weights.best.cntk.hdf5
16136997/16136997 - 41s - loss: 9.8225e-05 - accuracy: 0.9935 - val_loss: 3.2176e-06 - val_accuracy: 0.9965
Epoch 2/6

Epoch 00002: val_loss did not improve from 0.00000
16136997/16136997 - 40s - loss: 1.0229e-05 - accuracy: 0.9963 - val_loss: 1.1411e-04 - val_accuracy: 0.9956
Epoch 3/6

Epoch 00003: val_loss did not improve from 0.00000
16136997/16136997 - 40s - loss: 2.1324e-05 - accuracy: 0.9957 - val_loss: 1.7143e-05 - val_accuracy: 0.9949
Epoch 4/6

Epoch 00004: val_loss did not improve from 0.00000
16136997/16136997 - 39s - loss: 3.0291e-05 - accuracy: 0.9954 - val_loss: 5.1116e-05 - val_accuracy: 0.9942
Epoch 5/6

Epoch 00005: val_loss did not improve from 0.00000
16136997/16136997 - 40s - loss: 2.0602e-05 - accuracy: 0.9953 - val_loss: 1.1229e-04 - val_accuracy: 0.9937
Epoch 6/6

Epoch 00006: val_loss di